In [3]:
pip install google-generativeai


In [17]:
import os
import sqlite3
import logging
from datetime import datetime
from openai import OpenAI


logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise EnvironmentError("Please set the OPENAI_API_KEY environment variable.")
client = OpenAI(api_key=OPENAI_API_KEY)

DB_PATH = "tutoring.db"
SUMMARY_DIR = "generated_summaries"
TRANSCRIPT_FILE = "Tutoring_Transcript_File.txt"  # Change if dynamic


os.makedirs(SUMMARY_DIR, exist_ok=True)

def initialize_db(conn):
    with conn:
        conn.executescript("""
        CREATE TABLE IF NOT EXISTS Students (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT
        );
        CREATE TABLE IF NOT EXISTS Tutors (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT
        );
        CREATE TABLE IF NOT EXISTS Sessions (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            student_id INTEGER,
            tutor_id INTEGER,
            date TEXT,
            transcript TEXT,
            FOREIGN KEY (student_id) REFERENCES Students(id),
            FOREIGN KEY (tutor_id) REFERENCES Tutors(id)
        );
        CREATE TABLE IF NOT EXISTS Summaries (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            session_id INTEGER,
            summary TEXT,
            created_at TEXT DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (session_id) REFERENCES Sessions(id)
        );
        """)

def get_or_create_id(conn, table, name):
    cur = conn.cursor()
    cur.execute(f"SELECT id FROM {table} WHERE name=?", (name,))
    row = cur.fetchone()
    if row:
        return row[0]
    cur.execute(f"INSERT INTO {table} (name) VALUES (?)", (name,))
    conn.commit()
    return cur.lastrowid

def add_session(conn, student_name, tutor_name, transcript, date_str):
    student_id = get_or_create_id(conn, "Students", student_name)
    tutor_id = get_or_create_id(conn, "Tutors", tutor_name)
    with conn:
        conn.execute("""
            INSERT INTO Sessions (student_id, tutor_id, date, transcript)
            VALUES (?, ?, ?, ?)
        """, (student_id, tutor_id, date_str, transcript))
    return conn.execute("SELECT last_insert_rowid()").fetchone()[0], student_id

def get_last_two_summaries(conn, student_id):
    cur = conn.cursor()
    cur.execute("""
        SELECT summary FROM Summaries
        JOIN Sessions ON Summaries.session_id = Sessions.id
        WHERE Sessions.student_id = ?
        ORDER BY Summaries.created_at DESC LIMIT 2
    """, (student_id,))
    return [row[0] for row in cur.fetchall()]

def generate_summary(previous_summaries, new_transcript):
    prompt = (
        "You are a professional academic tutor.\n"
        "Below are prior summaries and a transcript of the new session.\n\n"
        "Previous summaries:\n" +
        ('---\n'.join(previous_summaries) if previous_summaries else 'None') +
        "\n\nTranscript:\n" + new_transcript +
        "\n\nWrite a warm, professional, parent-facing summary of this new tutoring session. "
        "Include what the student worked on, what they did well, what they struggled with, and any next steps."
    )
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

def save_summary_to_file(summary_text, transcript_filename, student_name):
    base_name = os.path.splitext(os.path.basename(transcript_filename))[0]
    out_path = os.path.join(SUMMARY_DIR, f"{student_name.replace(' ', '_')}_{base_name}_summary.txt")
    with open(out_path, "w") as f:
        f.write(summary_text)
    logging.info(f"Summary saved to {out_path}")

def load_transcript(filepath):
    with open(filepath, "r") as f:
        return f.read()

def main():
    transcript_text = load_transcript(TRANSCRIPT_FILE)
    student_name = "Jane Doe"
    tutor_name = "Mr. Smith"
    date = datetime.now().isoformat()

    with sqlite3.connect(DB_PATH) as conn:
        initialize_db(conn)
        session_id, student_id = add_session(conn, student_name, tutor_name, transcript_text, date)
        last_summaries = get_last_two_summaries(conn, student_id)
        new_summary = generate_summary(last_summaries, transcript_text)

        conn.execute("INSERT INTO Summaries (session_id, summary) VALUES (?, ?)",
                     (session_id, new_summary))
        conn.commit()

        save_summary_to_file(new_summary, TRANSCRIPT_FILE, student_name)
        logging.info("New summary generated successfully:\n\n" + new_summary)

if __name__ == "__main__":
    main()
